In [13]:
import pandas as pd
from tqdm import tqdm
import json
import requests

## Get Access Token

In [ ]:
# 쇼핑몰 : 오드

data = {
    "grant_type" : "client_credentials",
    "client_id" : '',
    "client_secret" : ''
}

# header = {'content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}
url = 'https://api.cre.ma/oauth/token'

token_response = requests.post(url, data=data)

token_response

In [ ]:
contents = json.loads(token_response.text)

ACCESS_TOKEN = contents['access_token']

ACCESS_TOKEN

## Get Reviews Datasets

#### parameter

```
- limit : (integer) 한 페이지에 가져올 리뷰의 개수입니다. 최대 100개까지 가능합니다. 기본: 30
- page : (integer) 페이지 번호입니다. 최대 페이지 수는 Link http header의 rel="last"에 기재되어 있습니다. 기본: 1
    - 최대 페이지 번호 알아오기
- product_id : (integer)특정 상품의 리뷰만 가져올 때 사용합니다.
- product_code : (string) 특정 상품의 리뷰만 가져올 때 사용합니다. 상품의 쇼핑몰 서비스측 고유 식별자.
- user_code : (string) 특정 회원의 리뷰만 가져올 때 사용합니다. 리뷰 작성자의 쇼핑몰 아이디.
    - None이 많다...?
- score : (integer) 특정 평점의 리뷰만 가져올 때 사용합니다. 1~5 값을 요청할 수 있습니다.
- photo : (integer) 포토리뷰만 가져오고 싶은 경우 1을 넘깁니다.

- date_updated_at : (integer) 리뷰의 생성 일자 대신 수정 일자를 기준으로 가져오고 싶은 경우 1을 넘깁니다.
- date_order_desc : (integer) 날짜기준 역순으로 가져오고 싶은 경우 1을 넘깁니다.
- start_date : (date) 입력하면 해당 날짜부터 작성된 리뷰만 가져오게 됩니다. 형식: "yyyy-mm-dd"
- end_date : (date) 입력하면 해당 날짜까지 작성된 리뷰만 가져오게 됩니다. 형식: "yyyy-mm-dd"
```

In [ ]:
def get_review_one_page(params, url, page=1):

    # 인자 값 체크
    # pass
    
    params['page'] = page
    
    response = requests.get(url, params=params)
    review_list = json.loads(response.text)
    
    return review_list

In [ ]:
def get_reviews(url=REVIEW_URL, access_token=ACCESS_TOKEN,
               limit=30, start_page=1, end_page=1,
               product_id=None, product_code=None,
               user_code=None,
               score=None,
               photo=None, 
               date_updated_at=None, date_order_desc=None,
               start_date=None, end_date=None,
               **kwargs):

    # 인자 값 체크
    # pass

    params = {
        'access_token' : ACCESS_TOKEN,
        "limit" : limit,
        'product_id' : product_id,
        "product_code" : product_code,
        'user_code' : user_code,
        'score' : score,
        'photo' : photo,
        'date_updated_at' : date_updated_at,
        'date_order_desc' : date_order_desc,
        'start_date' : start_date,
        'end_date' : end_date,
    }
    
    review_list = []
    
    for page in tqdm(range(start_page, end_page + 1)):
        review_one_page = get_review_one_page(params, url, page)
        review_list.extend(review_one_page)
        time.sleep(0.1)
        
    return review_list

In [34]:
REVIEW_URL = "http://api.cre.ma/v1/reviews"

params = {
    'access_token' : ACCESS_TOKEN,
    "limit" : 100,
}

response = requests.get(REVIEW_URL, params=params)

response.headers['Link']

'<https://api.cre.ma/v1/reviews?access_token=im75d0sXrsUe9S5eZBmG-PfFNMs97hmiJDgw1VODa1U&limit=100&page=5454>; rel="last", <https://api.cre.ma/v1/reviews?access_token=im75d0sXrsUe9S5eZBmG-PfFNMs97hmiJDgw1VODa1U&limit=100&page=2>; rel="next"'

In [35]:
PRODUCT_URL = "http://api.cre.ma/v1/products"

params = {
    'access_token' : ACCESS_TOKEN,
    "limit" : 100,
}

response = requests.get(PRODUCT_URL, params=params)

response.headers['Link']

'<https://api.cre.ma/v1/products?access_token=im75d0sXrsUe9S5eZBmG-PfFNMs97hmiJDgw1VODa1U&limit=100&page=378>; rel="last", <https://api.cre.ma/v1/products?access_token=im75d0sXrsUe9S5eZBmG-PfFNMs97hmiJDgw1VODa1U&limit=100&page=2>; rel="next"'

In [ ]:
REVIEW_URL = "http://api.cre.ma/v1/reviews"

# 2019년 이후 데이터
params = {
    'access_token' : ACCESS_TOKEN,
    "limit" : 100,
    'start_page' : 1,
    'end_page' : 1110,
    'product_id' : None,
    "product_code" : None,
    'user_code' : None,
    'score' : None,
    'photo' : None,
    'date_updated_at' : None,
#     'date_order_desc' : 1,
    'start_date' : '2019-01-01',
#     'end_date' : '2021-02-06',
    
}

review_list = get_reviews(**params)

review_df = pd.DataFrame(review_list)

print(review_df.shape)
review_df.head()

In [ ]:
review_df.to_csv("review_since_20190101.csv", index=False)

## Get Products Data

In [1]:
def get_product_one_page(params, url, page=1):

    # 인자 값 체크
    # pass
    
    params['page'] = page
    
    response = requests.get(url, params=params)
    product_list = json.loads(response.text)
    
    return product_list

In [ ]:
def get_products(url=PRODUCTS_URL, access_token=ACCESS_TOKEN,
               limit=30, start_page=1, end_page=1,
               **kwargs):

    # 인자 값 체크
    # pass

    params = {
        'access_token' : ACCESS_TOKEN,
        "limit" : limit,
    }
    
    product_list = []
    
    for page in tqdm(range(start_page, end_page + 1)):
        product_one_page = get_product_one_page(params, url, page)
        product_list.extend(product_one_page)
        time.sleep(0.1)
        
    return product_list

In [ ]:
PRODUCTS_URL = "https://api.cre.ma/v1/products"

params = {
    'access_token' : ACCESS_TOKEN,
    "limit" : 100,
    'end_page' : 372,
}

product_list = get_products(**params)

products_df = pd.DataFrame(product_list)

print(products_df.shape)
products_df.head()

In [ ]:
products_df.to_csv("data/products.csv", index=False)

In [ ]:
# cols = ['name', 'code', 'created_at', 'updated_at', 'product_status']
# # drop_cols = ["id", 'created_at']
# products_selected = products[cols].copy()

# print(products_selected.shape)
# products_selected.head()